# BackTesting 


In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv('BTC-USD.csv', parse_dates=['Date'], index_col=['Date'])
data = data.loc['2020-01-01':]
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,7200.174316,18565664997
2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,6985.470215,20802083465
2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,7344.884277,28111481032
2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,7410.656738,18444271275
2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,7411.317383,19725074095
...,...,...,...,...,...,...
2024-12-18,106147.296875,106470.609375,100041.539062,100041.539062,100041.539062,93865656139
2024-12-19,100070.687500,102748.148438,95587.679688,97490.953125,97490.953125,97221662392
2024-12-20,97484.695312,98098.914062,92175.179688,97755.929688,97755.929688,105634083408


In [3]:
from backtesting import Strategy, Backtest
from backtesting.lib import crossover, plot_heatmaps
import talib 

class SMAcross(Strategy):
    def init(self):
        self.ema = self.I(talib.EMA, self.data.Close, timeperiod=20)
                
    def next(self): # Se llama en cada paso o dia.
        price = self.data.Close[-1] # Precio de cierre del ultimo dia, ya que viene el acumulado por cada dia que pasa.

        if price > self.ema[-1] and not self.position.is_long: 
            # self.buy(size=10, tp=price*1.3, sl=price*0.85)
            self.buy(sl=price*0.95)
        elif price <= self.ema[-1] and self.position.is_long:
            self.position.close()

bt = Backtest(data, SMAcross,cash=200000 )
bt.run()

Start                     2020-01-01 00:00:00
End                       2024-12-22 00:00:00
Duration                   1817 days 00:00:00
Exposure Time [%]                   59.735974
Equity Final [$]               1871296.116309
Equity Peak [$]                2035758.335059
Return [%]                         835.648058
Buy & Hold Return [%]             1223.254205
Return (Ann.) [%]                   56.664406
Volatility (Ann.) [%]               69.053085
Sharpe Ratio                         0.820592
Sortino Ratio                        2.152484
Calmar Ratio                         1.110055
Max. Drawdown [%]                    -51.0465
Avg. Drawdown [%]                   -7.418932
Max. Drawdown Duration      860 days 00:00:00
Avg. Drawdown Duration       42 days 00:00:00
# Trades                                  101
Win Rate [%]                        27.722772
Best Trade [%]                      67.130993
Worst Trade [%]                     -5.042587
Avg. Trade [%]                    

In [4]:
bt.plot()

/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest valu

GridPlot(id='p1320', ...)

In [ ]:
class RSI(Strategy):
    rsi_period = 22 # atributo de clase
    sl_percent = 0.95
    out_position = 30
    def init(self):
        self.rsi = self.I(talib.RSI, self.data.Close, timeperiod=self.rsi_period) # accedo al atributo de la clase desde una instancia. 
        
    def next(self):
        price = self.data.Close[-1]

        if self.rsi[-1] < self.out_position:
            self.position.close()

        if self.position.is_long:
            return
        
        self.buy(sl= price*self.sl_percent)

bt = Backtest(data, RSI, cash=200000)
bt.optimize(rsi_period=range(5, 30, 1),
            sl_percent=[0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99],
            maximize='Return [%]')

TypeError: unhashable type: 'numpy.ndarray'

In [8]:
bt.plot()

/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest valu

GridPlot(id='p1705', ...)